In [ ]:
import numpy as np
import time
import pandas as pd
import openpyxl
import os
from qiskit import QuantumCircuit, transpile

from Noise import QiskitNoiseModelDepol,QiskitNoiseModelBitflip, QiskitNoiseModelAmplitudeDamp
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Operator
from qiskit.utils.parallel import parallel_map
from qiskit_aer.noise import NoiseModel

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeAthensV2

#
# Helper functions
#
def groverSuboptimalIterations(n, k):
    num= int(np.round(0.58278 * np.sqrt(n / k)))
    if num<1:
        return 1
    else:
        return num


# n is the size of the database, m is the number of marked items
def grover_iterations(m, n):
    #return int(np.round(np.pi *(np.sqrt(n)/4)))

    num =int(np.round(np.pi / (4 * np.arcsin(np.sqrt(m / n))) - 1 / 2)) #optimistic grover

    if num<1:
        return 1
    else:
        return num

# selects n unique random numbers from in the range [0 - upper_bound)
def select_random_elements(upper_bound, n):
    elements = list(range(upper_bound))
    selected = []
    for i in range(n):
        index = np.random.randint(len(elements))
        selected.append(elements[index])
        del elements[index]
    return selected

# Checks if the integer representetion of binary_string exists in number_list
def binary_string_in_list(number_list, binary_string):
    num = int(binary_string, 2)
    return num in number_list

#
# Grover implementation
#

def phase_oracle(n, indicies_to_mark, name='Oracle'):
    qc = QuantumCircuit(n, name=name)
    oracle_matrix = np.identity(2**n)
    for index_to_mark in indicies_to_mark:
        oracle_matrix[index_to_mark, index_to_mark] = -1
    qc.unitary(Operator(oracle_matrix), range(n))
    return qc


# Dynamic programming optimization of diffuser function
# dict to save dicts that have been computed 
diffuser_dict = {}

def diffuser(n):
    if n not in diffuser_dict:
        qc = QuantumCircuit(n, name='Diffuser')
        qc.h(range(n))
        qc.append(phase_oracle(n, [0]), range(n))
        qc.h(range(n))
        diffuser_dict.update({n: qc})
    return diffuser_dict.get(n)


def Grover(n, marked, r):
    # Reuse the same oracle instance rather than creating new ones
    qc_phase = phase_oracle(n, marked)
    qc_diffuser = diffuser(n)
    
    qc = QuantumCircuit(n, n)
    qc.h(range(n))
    for _ in range(r):
        qc.append(qc_phase, range(n)) 
        qc.append(qc_diffuser, range(n))
    qc.measure(range(n), range(n))
    return qc

#
# Functions to run experiments
#
def simulator_experiment_noise(n, noise_level):


    nums = range(24, int(np.floor((2**n) / 2)) + 1)

    
    for num in nums:
        result = experiment_parallel_noise(num, n, noise_level)
        saving_data_single(n, noise_level, num, result)

    # Fewer processes for larger circuits to reduce memory contention
    #num_processes = max(1, 4 - (n - 3))  # Scale down as n increases
    #results = parallel_map(experiment_parallel_noise, nums, task_args=([n, noise_level]), num_processes=num_processes)
    #saving_data(n, noise_level, results)

def saving_data_single(n, noise_level, num_marked_items, result):
    actual_prob = result[0]
    oracle_call = result[1]

    noise_type = "athens"  # Adjust this if needed
    excel_filename = f"./noise_{noise_type}_1_to_5qubit_nogrant_opt.xlsx"

    sheet_prob = f"{n} qubits"
    sheet_oracle = f"{n}qbit_oracle"

    if os.path.exists(excel_filename):
        with pd.ExcelWriter(excel_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            try:
                prob_df = pd.read_excel(excel_filename, sheet_name=sheet_prob)
            except:
                prob_df = pd.DataFrame()

            try:
                oracle_df = pd.read_excel(excel_filename, sheet_name=sheet_oracle)
            except:
                oracle_df = pd.DataFrame()

            # Ensure the dataframes have the right index (based on Number of Marked Items)
            if "Number of Marked Items" not in prob_df.columns:
                prob_df["Number of Marked Items"] = []
            if "Number of Marked Items" not in oracle_df.columns:
                oracle_df["Number of Marked Items"] = []

            prob_df = prob_df.set_index("Number of Marked Items")
            oracle_df = oracle_df.set_index("Number of Marked Items")

            prob_df.loc[num_marked_items, f"{noise_level} noise level"] = actual_prob
            oracle_df.loc[num_marked_items, f"{noise_level} noise level oracle calls"] = oracle_call

            prob_df = prob_df.reset_index()
            oracle_df = oracle_df.reset_index()

            prob_df.to_excel(writer, index=False, sheet_name=sheet_prob)
            oracle_df.to_excel(writer, index=False, sheet_name=sheet_oracle)
    else:
        # First-time creation
        prob_df = pd.DataFrame({
            "Number of Marked Items": [num_marked_items],
            f"{noise_level} noise level": [actual_prob]
        })
        oracle_df = pd.DataFrame({
            "Number of Marked Items": [num_marked_items],
            f"{noise_level} noise level oracle calls": [oracle_call]
        })

        with pd.ExcelWriter(excel_filename, engine='openpyxl', mode='w') as writer:
            prob_df.to_excel(writer, index=False, sheet_name=sheet_prob)
            oracle_df.to_excel(writer, index=False, sheet_name=sheet_oracle)

    print(f"✔️ Saved result for marked items={num_marked_items} at noise level={noise_level}")

def experiment_parallel_noise(nums, n, noise_level):
    print(f"started job {nums}")

    # how many grover iterations should be performed
    iterations = grover_iterations(nums, 2**n) #optimistic grover
    #iterations = groverSuboptimalIterations(2**n,nums) #pesemistisk grover

    fake_backend = FakeAthensV2()
    noise_model = NoiseModel.from_backend(fake_backend)



    #noise_instance = QiskitNoiseModelAmplitudeDamp(noise_level)
    #noise_instance = QiskitNoiseModelBitflip(noise_level)
    #noise_instance = QiskitNoiseModelDepol(noise_level)

   # noise_model = noise_instance.get_noise_model()
    sim = AerSimulator(noise_model=noise_model, shots=1000)


    iterations_per_marked_item_set = 100
    shots = 1000

    hits = 0
    for j in range(0,iterations_per_marked_item_set):

        marked = select_random_elements(2**n, nums)
        qc = Grover(n, marked, iterations)
        t_qc = transpile(qc, sim) 
        result = sim.run(t_qc, memory=True).result()
        counts = result.get_memory()
        for count in counts:
            found_correct_element = binary_string_in_list(marked, count)
            if found_correct_element:
                hits += 1
        print(f">>> Job {nums} finished iteration {j + 1} / {iterations_per_marked_item_set}")

    return ( ((hits / (iterations_per_marked_item_set * shots)) * 100), ((shots *iterations_per_marked_item_set)/ hits)*(iterations + 1))

#simulator_experiment(4)

#simulator_experiment_noise(3, 0.000)

#if __name__ == "__main__":
 #       for i in range(1, 11):
  #          simulator_experiment_noise(5, 0.0005*i)


#if __name__ == "__main__":
 #   for j in range(2, 7):
  #          simulator_experiment_noise(j, 0)
        
if __name__ == "__main__":
    for j in range(6, 7):
        for i in range(10, 11):
            simulator_experiment_noise(j, 0.0005*i)




#TESTs: med qbits 2 till 6
# inget noise
#testa fake backend Marrakech
#noise med marrakesh's min max (0.001, 0.003) vilket vi kör mellan 0.0005 och 0.005 ta 10 steg
#gör det för både optimistisk och pessimistisk grover

#antal test 8*(1+1+10)*2 = 176

#grafer:
# alla qbits
#optimistic grover, pessimistic grover:
#(ingetnoise, marrakesh, clasikal, min, max)

# antal grafer = 8 * 2  = 16
#2,4,8,16,32
#tabbeller för datan för noise

started job 24
>>> Job 24 finished iteration 1 / 100
>>> Job 24 finished iteration 2 / 100
>>> Job 24 finished iteration 3 / 100
>>> Job 24 finished iteration 4 / 100
>>> Job 24 finished iteration 5 / 100
>>> Job 24 finished iteration 6 / 100
>>> Job 24 finished iteration 7 / 100
>>> Job 24 finished iteration 8 / 100
>>> Job 24 finished iteration 9 / 100
>>> Job 24 finished iteration 10 / 100
>>> Job 24 finished iteration 11 / 100
>>> Job 24 finished iteration 12 / 100
>>> Job 24 finished iteration 13 / 100
>>> Job 24 finished iteration 14 / 100
>>> Job 24 finished iteration 15 / 100
>>> Job 24 finished iteration 16 / 100
>>> Job 24 finished iteration 17 / 100
>>> Job 24 finished iteration 18 / 100
>>> Job 24 finished iteration 19 / 100
>>> Job 24 finished iteration 20 / 100
>>> Job 24 finished iteration 21 / 100
>>> Job 24 finished iteration 22 / 100
>>> Job 24 finished iteration 23 / 100
>>> Job 24 finished iteration 24 / 100
>>> Job 24 finished iteration 25 / 100
>>> Job 24 finished